# <center>Lab Two Classification</center>
<center><font size = "4"> 2017-2018 California Department of Education Mathmematics Achievement</font></center>

##### <center>Create by An Nguyen, Andy Ho, Jodi Pafford</center>
<center> Februar 27, 2019</center>

## Data Preparation (15 points total)

<p>&nbsp;&nbsp;&nbsp;&nbsp;[10 points] Deﬁne and prepare your class variables. Use proper variable representations (int, ﬂoat, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.</p>

<p>&nbsp;&nbsp;&nbsp;&nbsp;[5 points] Describe the ﬁnal dataset that is used for classiﬁcation/regression (include a description of any newly formed variables you created).</p>

## Modeling and Evaluation (70 points total)

<p>&nbsp;&nbsp;&nbsp;&nbsp;[10 points] Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.</p>

<p>&nbsp;&nbsp;&nbsp;&nbsp;[10 points] Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratiﬁed 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.</p>

<p>&nbsp;&nbsp;&nbsp;&nbsp;[20 points] Create three different classiﬁcation/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!</p> 

<p>&nbsp;&nbsp;&nbsp;&nbsp;[10 points] Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model. 

<p>&nbsp;&nbsp;&nbsp;&nbsp;[10 points] Discuss the advantages of each model for each classiﬁcation task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference signiﬁcant with 95% conﬁdence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.</p> 

<p>&nbsp;&nbsp;&nbsp;&nbsp;[10 points] Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classiﬁcation task.</p>

## Deployment (5 points total) 

<p>&nbsp;&nbsp;&nbsp;&nbsp;How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?</p>

## Exceptional Work (10 points total) 

<p>&nbsp;&nbsp;&nbsp;&nbsp;You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most signiﬁcant for making a good model for each classiﬁcation algorithm? </p>